In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Libraries and packages

**Torch, torch audio, torchvision**: this installs PyTorch, the primary deep learning framework used to build, train, and run neural networks. The --index-url pointing to cu126 ensures get a version compatible with NVIDIA GPUs using CUDA 12.6, which is crucial for hardware acceleration.sential for efficient numerical computations (arrays, matrices), while** Pandas** is used for data manipulation and analysis (like handling dataframes).** Transformers**: This is the core library,it provides thousands of pre-trained models (like Llama 3), along with the code and tools needed to use them for training and inference. **PEFT**: It implements methods like LoRA (Low-Rank Adaptation), which the paper explicitly uses to fine-tune Llama3-8B by only training a small fraction of the model's parameters. **Accelerate** : This library simplifies running PyTorch code across different hardware setups (e.g., single GPU, multiple GPUs, or TPUs) with minimal code changes, making training more efficient.**trl**:it includes easy-to-use tools for training LLMs, especially for tasks like Supervised Fine-Tuning (SFT), which was a key method used in the paper to train models on the Knights and Knaves puzzles. **Datasets**: A library for easily downloading, processing, and managing large datasets for training and evaluation.



In [ ]:
# STEP 1: Upgrade core libraries and install PyTorch with CUDA support
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
!pip install --upgrade numpy pandas

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.9/821.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 4.0 MB/s

In [ ]:

# STEP 2: Install the transformer stack and other tools
!pip install --upgrade transformers peft accelerate bitsandbytes trl datasets tiktoken wandb xformers optimum scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.8/425.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install  flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 72.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.2-cp311-cp311-linux_x86_64.whl size=255941375 sha256=bf8e2091a699751a2311586c0a26f49e6ee6ed305ab85d0d50873f747e27cbab
  Stored in directory: /root/.cache/pip/wheels/a1/67/59/31ba3e82c0d8ff661e94711512b3e41e07ac561aed265acb72
Successfully built flash-attn


In [ ]:
import os

In [ ]:
os.makedirs('/content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs', exist_ok=True)

In [ ]:
# Helpful suggestion: code with %% would just update the files not remove them or create another one.

In [ ]:
%%writefile /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/requirements.txt
transformers==4.38.2
torch==2.2.1
datasets==2.18.0
accelerate==0.28.0
pandas==2.2.1
numpy==1.26.4
scikit-learn==1.4.1
peft==0.10.0
bitsandbytes==0.43.0
trl==0.8.6
tiktoken==0.7.0
wandb==0.17.0
xformers==0.0.25.post1
optimum==1.19.1
flash-attn==2.5.8

Overwriting /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/requirements.txt


In [ ]:
%%writefile /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/README.md

Overwriting /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/README.md


# Datasets - Train and Test, Data Generation


In [ ]:
# import json : Loads Python's built-in module to handle JSON data.
# import glob: Allows the user to find file paths using wildcards (e.g., "*.jsonl").

# function for loading jsonl file, reading them and closing them after being read
def load_jsonl(filepath):
    with open(filepath, 'r') as file:
        return [json.loads(line) for line in file]

we will remove the test and train data from here

In [ ]:
# # load train and test data

# train_clean_files = glob.glob("/content/drive/MyDrive/Code-Release-KK/data/train/clean/*.jsonl")
# train_data = []
# for f in train_clean_files:
#     train_data.extend(load_jsonl(f))

# # Load test data
# test_clean_files = glob.glob("/content/drive/MyDrive/Code-Release-KK/data/test/clean/*.jsonl")
# test_data = []
# for f in test_clean_files:
#     test_data.extend(load_jsonl(f))

# Reading Prompt.py file - These contain the templates used by the data_gen.py file.

In [ ]:
%%writefile /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/prompt.py
# PASTE THE ENTIRE CONTENT OF YOUR ORIGINAL PROMPT.PY FILE HERE
# (It starts with system_instruction=''' and ends with the last ''')

system_instruction='''Your task is to solve a logical reasoning problem. You are given set of statements from which you must logically deduce the identity of a set of characters.

You must infer the identity of each character. First, explain your reasoning. At the end of your answer, you must clearly state the identity of each character by following the format:

CONCLUSION:
(1) ...
(2) ...
(3) ...
'''


system_instruction_no_reason='''Your task is to solve a logical reasoning problem. You are given set of statements from which you must logically deduce the identity of a set of characters.

You must infer the identity of each character. At the end of your answer, you must clearly state the identity of each character by following the format:

CONCLUSION:
(1) ...
(2) ...
(3) ...
'''

demonstration_2char_no_reason='''### Question: A very special island is inhabited only by knights and knaves. Knights always tell the truth, and knaves always lie. You meet 2 inhabitants: Jack, and Sophia. Jack tells you that Sophia is not a knave. Sophia says that If Jack is a knight then Sophia is a knight. So who is a knight and who is a knave?
### Answer:
CONCLUSION:
(1) Jack is a knight
(2) Sophia is a knight
'''



demonstration_2char='''### Question: A very special island is inhabited only by knights and knaves. Knights always tell the truth, and knaves always lie. You meet 2 inhabitants: Ella, and Penelope. In a statement by Ella: \"Ella is a knight or Penelope is a knight\". According to Penelope, \"Ella is a knave if and only if Penelope is a knight\". So who is a knight and who is a knave?
### Answer: Let's think step by step, by considering whether each person is lying and if that leads to contradiction. Assume Ella is a knight. Penelope cannot be a knight, because this would contradict the claim of their own. Penelope cannot be a knave, because this would contradict the false claim of their own. We have exhausted all possibilities for Penelope, so let us go back and reconsider Ella. Assume Ella is a knave. Penelope cannot be a knight, because this would contradict the false claim of Ella. Assume Penelope is a knave. This leads to a feasible solution.
CONCLUSION:
(1) Ella is a knave
(2) Penelope is a knave
'''

Overwriting /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/prompt.py


In [ ]:
import os
project_path = "/content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/"
os.chdir(project_path)
print(f"Changed current directory to: {os.getcwd()}")

Changed current directory to: /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs


In [ ]:
!python /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/data_gen_kk.py

In [ ]:
!pwd

/content


In [ ]:
!ls -l data/

ls: cannot access 'data/': No such file or directory


Fixed the Index error, we will not use the data for train and test by author, now we move onto next step which is Fine tuning but before that we login in to Hugging face

We use Hugging Face to load pre-trained causal language models and their tokenizers from the Hugging Face Hub.

# Fine-Tuning

We access Hugging Face Transformers,

In [ ]:

from huggingface_hub import notebook_login
notebook_login()

Weights and biases: Its primary purpose is authentication, verifying your identity to ensure that sensitive experiment data is kept private. Once authenticated, the key authorizes your script to log a continuous stream of information, such as performance metrics, hyperparameters, and system usage, directly to your project dashboard.

In [ ]:
!wandb login

wandb: Currently logged in as: jrashi (jrashi-boston-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Using University id for this.
YOUR_WANDB_API_KEY = "WANDBKEY" # <<< real key replaced with none working code

import os
os.environ["WANDB_API_KEY"] = YOUR_WANDB_API_KEY

print("W&B API key environment variable set.")

W&B API key environment variable set.


In [ ]:
# checking if hf.py is in Google drive, if yes, then we read it and modify to proceed to fine-tuning.

In [ ]:
import os
# Ensure we are in the correct directory first
project_path = "/content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs"
os.chdir(project_path)
print(f"Current working directory: {os.getcwd()}")

# List all files and folders in the current directory
print("\nContents of the current directory:")
!ls -l

Current working directory: /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs

Contents of the current directory:
total 1186
-rw------- 1 root root    2137 Jul 10 03:06 '=0.29.0'
-rw------- 1 root root     887 Jun 29 19:51  base.py
drwx------ 3 root root    4096 Jun 26 17:21  data
-rw------- 1 root root   17488 Jun 27 00:50  data_gen_kk.py
drwx------ 2 root root    4096 Jun 26 17:21  dataset
-rw------- 1 root root    8443 Aug  6 14:29  eval_kk.py
-rw------- 1 root root   12627 Jun 28 03:15  finetune_kk.py
-rw------- 1 root root   17700 Jul 10 14:41  hf.py
-rw------- 1 root root       0 Jan  1  1980  __init__.py
-rw------- 1 root root    6204 Jun 29 20:16  kk.py
-rw------- 1 root root   41754 Jun 27 00:30  lib_kk.py
-rw------- 1 root root    7201 Jan  1  1980  mem_classifier_model.py
-rw------- 1 root root    8799 Jan  1  1980  mem_classifier_puzzle.py
-rw------- 1 root root 1026575 Jun 26 17:42  MEM_RSN_LLM_06112025-2025.ipynb
drwx------ 2 root root    4096 Jun 26 18:07  merged_phi3_direct
-r

In [ ]:
hf_file_path = "hf.py"

Hf.py needs to be edited at specific places to include Phi-3-mini-4k-instruct the open source llm we have chosen for this model run(Model- path is modified), and same goes for sh. scripts because we are only using Phi-3-mini-instruct.

run_train_0shot.sh: need explicit mention of file path for intermediate fine tuned model

run_train_1shot.sh -need explicit mention of file path for intermediate fine tuned model

run_train_pertub.sh need explicit mention of file path for intermediate fine tuned model at FINETUNED-MODEL-PATH

run_test.sh : models=()

run_test_perturb.sh : models=()

run_test.sh and run_test_perturb.sh, we change num_limit, models=( ... ) array:max_token, --batch_size:, --cot flag for ,--use_vllm,Specific Llama-3 Block  we can comment it out if we are not using it.

run_test_perturb.sh specifically: --problem_type, This argument (e.g., perturbed_leaf, perturbed_statement) specifies the type of perturbed Knight and Knave problems to evaluate.

finetune_kk.py: need model path-- os.directory for storing the model weight and tokenizer.
hf.py: def query_generation(self, prompt will require change in LLM based on the type of LLM to be chosen, Choose the correct fine_tuned_adapter_path based on your fine-tuning option (Direct or CoT), this also needs to be chnaged accordingly.

Hf.py was edited, now need to fine_tune_kk with specific settings, need ot choose which method to apply direct or cot.


We run the Direct fine-tuning part first, we need to change the test, train, outputdirectory path, run_name for every run for 2,3,4,5,6,7,8. Then we run evaluation for each 2* 2,3,4,5,6,7,8, we do need to do evaluate the model with adaper weights, then we merge the adapter weight to get merged model.

# Direct Fine-Tuning

## Fine-tuning: Direct method: 2 people puzzle

In [ ]:

!python finetune_kk.py \
    --train_data ./data/train/clean/people2_num200.jsonl \
    --test_data ./data/test/clean/people2_num100.jsonl \
    --model_checkpoint microsoft/Phi-3-mini-4k-instruct \
    --output_dir ./Models/Intermd/Phi3/Direct/finetuned_people2 \
    --num_train_epochs 10 \
    --save_strategy steps \
    --save_steps 0.2 \
    --max_seq_length 512 \
    --eval_steps 5 \
    --project_name my_kk_repro_phi3 \
    --run_name phi3_direct_ft_people2 \
    --wandb_key "8d6f5281721a0539256b8890ff0696e1c6f911c2"\
    --add_eos




## Fine-tuning: Direct method: 3 people puzzle

In [ ]:
!python finetune_kk.py \
    --train_data ./data/train/clean/people3_num1000.jsonl \
    --test_data ./data/test/clean/people3_num100.jsonl \
    --model_checkpoint microsoft/Phi-3-mini-4k-instruct \
    --output_dir ./Models/Intermd/Phi3/Direct/finetuned_people3 \
    --num_train_epochs 10 \
    --save_strategy steps \
    --save_steps 0.2 \
    --max_seq_length 512 \
    --eval_steps 5 \
    --project_name my_kk_repro_phi3 \
    --run_name phi3_direct_ft_people3 \
    --wandb_key "8d6f5281721a0539256b8890ff0696e1c6f911c2"\
    --add_eos

## Fine-tuning: Direct method: 4 people puzzle

In [ ]:
!python finetune_kk.py \
    --train_data ./data/train/clean/people4_num1000.jsonl \
    --test_data ./data/test/clean/people4_num100.jsonl \
    --model_checkpoint microsoft/Phi-3-mini-4k-instruct \
    --output_dir ./Models/Intermd/Phi3/Direct/finetuned_people4 \
    --num_train_epochs 10 \
    --save_strategy steps \
    --save_steps 0.2 \
    --max_seq_length 512 \
    --eval_steps 5 \
    --project_name my_kk_repro_phi3 \
    --run_name phi3_direct_ft_people4 \
    --wandb_key "8d6f5281721a0539256b8890ff0696e1c6f911c2"\
    --add_eos

## Fine-tuning: Direct method: 5 people puzzle

In [ ]:
!python finetune_kk.py \
    --train_data ./data/train/clean/people5_num1000.jsonl \
    --test_data ./data/test/clean/people5_num100.jsonl \
    --model_checkpoint microsoft/Phi-3-mini-4k-instruct \
    --output_dir ./Models/Intermd/Phi3/Direct/finetuned_people5 \
    --num_train_epochs 10 \
    --save_strategy steps \
    --save_steps 0.2 \
    --max_seq_length 512 \
    --eval_steps 5 \
    --project_name my_kk_repro_phi3 \
    --run_name phi3_direct_ft_people5 \
    --wandb_key "8d6f5281721a0539256b8890ff0696e1c6f911c2"\
    --add_eos

## Fine-tuning: Direct method: 6 people puzzle

In [ ]:
!python finetune_kk.py \
    --train_data ./data/train/clean/people6_num1000.jsonl \
    --test_data ./data/test/clean/people6_num100.jsonl \
    --model_checkpoint microsoft/Phi-3-mini-4k-instruct \
    --output_dir ./Models/Intermd/Phi3/Direct/finetuned_people6 \
    --num_train_epochs 10 \
    --save_strategy steps \
    --save_steps 0.2 \
    --max_seq_length 512 \
    --eval_steps 5 \
    --project_name my_kk_repro_phi3 \
    --run_name phi3_direct_ft_people6 \
    --wandb_key "8d6f5281721a0539256b8890ff0696e1c6f911c2"\
    --add_eos


## Fine-tuning: Direct method: 7 people puzzle

In [ ]:
!python finetune_kk.py \
    --train_data ./data/train/clean/people7_num1000.jsonl \
    --test_data ./data/test/clean/people7_num100.jsonl \
    --model_checkpoint microsoft/Phi-3-mini-4k-instruct \
    --output_dir ./Models/Intermd/Phi3/Direct/finetuned_people7 \
    --num_train_epochs 10 \
    --save_strategy steps \
    --save_steps 0.2 \
    --max_seq_length 512 \
    --eval_steps 5 \
    --project_name my_kk_repro_phi3 \
    --run_name phi3_direct_ft_people7 \
    --wandb_key "8d6f5281721a0539256b8890ff0696e1c6f911c2"\
    --add_eos

## Fine-tuning: Direct method: 8 people puzzle

In [ ]:
!python finetune_kk.py \
    --train_data ./data/train/clean/people8_num1000.jsonl \
    --test_data ./data/test/clean/people8_num100.jsonl \
    --model_checkpoint microsoft/Phi-3-mini-4k-instruct \
    --output_dir ./Models/Intermd/Phi3/Direct/finetuned_people8 \
    --num_train_epochs 10 \
    --save_strategy steps \
    --save_steps 0.2 \
    --max_seq_length 512 \
    --eval_steps 5 \
    --project_name my_kk_repro_phi3 \
    --run_name phi3_direct_ft_people8 \
    --wandb_key "8d6f5281721a0539256b8890ff0696e1c6f911c2"\
    --add_eos

# Chain-of-thought Fine-Tuning


Chain-of-thought Fine-tuning Process: CoT is presented as a superior fine-tuning strategy for logical reasoning tasks like Knights and Knaves, as it pushes the model beyond mere input-output mapping to implicitly or explicitly learn the underlying reasoning process,

COT: 2 people Puzzle

In [ ]:
!python finetune_kk.py \
    --train_data ./data/train/clean/people2_num200.jsonl \
    --test_data ./data/test/clean/people2_num100.jsonl \
    --model_checkpoint microsoft/Phi-3-mini-4k-instruct \
    --output_dir ./Models/Intermd/Phi3/COT/finetuned_people2 \
    --cot_ft \
    --num_train_epochs 10 \
    --save_strategy steps \
    --save_steps 0.2 \
    --max_seq_length 512 \
    --eval_steps 5 \
    --project_name my_kk_repro_phi3 \
    --run_name phi3_cot_ft_people2\
    --wandb_key "8d6f5281721a0539256b8890ff0696e1c6f911c2"\
    --add_eos

Automated COT for N-2,3,4,5,6,7,8

In [ ]:
import os
project_path = '/content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/'
os.chdir(project_path)
print(f"Current working directory: {os.getcwd()}")

Current working directory: /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs


In [ ]:

%%writefile run_all_cot_finetuning.sh
#!/bin/bash

# Ensure the script stops if any command fails
set -e

# Set your Weights & Biases API Key here
export WANDB_API_KEY="8d6f5281721a0539256b8890ff0696e1c6f911c2"

# Base model checkpoint
MODEL_CHECKPOINT="microsoft/Phi-3-mini-4k-instruct"

# Base output directory structure for all fine-tuned models
BASE_OUTPUT_PARENT_DIR="./Models/Intermd/Phi3/COT"

# Ensure the base output parent directory exists
mkdir -p $BASE_OUTPUT_PARENT_DIR

# Define the N values to iterate through
N_VALUES=(2 3 4 5 6 7 8)
TRAIN_COUNTS=(200 1000 1000 1000 1000 1000 1000) # Corresponds to N_VALUES
TEST_COUNT=100

# Loop through each N-person puzzle
for i in "${!N_VALUES[@]}"; do
    N=${N_VALUES[$i]}
    TRAIN_NUM=${TRAIN_COUNTS[$i]}

    echo "--------------------------------------------------------"
    echo "Starting CoT Fine-tuning for ${N}-person puzzles..."
    echo "--------------------------------------------------------"

    TRAIN_DATA="./data/train/clean/people${N}_num${TRAIN_NUM}.jsonl"
    TEST_DATA="./data/test/clean/people${N}_num${TEST_COUNT}.jsonl"
    OUTPUT_DIR="${BASE_OUTPUT_PARENT_DIR}/finetuned_people${N}"
    RUN_NAME="phi3_cot_ft_people${N}"

    # Ensure this specific output directory exists
    mkdir -p $OUTPUT_DIR

    python finetune_kk.py \
        --train_data "$TRAIN_DATA" \
        --test_data "$TEST_DATA" \
        --model_checkpoint "$MODEL_CHECKPOINT" \
        --output_dir "$OUTPUT_DIR" \
        --cot_ft \
        --num_train_epochs 10 \
        --save_strategy steps \
        --save_steps 0.2 \
        --max_seq_length 512 \
        --eval_steps 5 \
        --project_name my_kk_repro_phi3 \
        --run_name "$RUN_NAME" \
        --add_eos \
        --fp16 \
        --per_device_train_batch_size 4

    echo "--------------------------------------------------------"
    echo "Finished CoT Fine-tuning for ${N}-person puzzles."
    echo "--------------------------------------------------------"
done

echo "All CoT fine-tuning jobs completed!"

Writing run_all_cot_finetuning.sh


In [ ]:
# code for running the script run_all_cot_finetuning which will run cot fine-tuning process for N: 2-8 people puzzle.

# Make the script executable
!chmod +x run_all_cot_finetuning.sh

# Run the script
!./run_all_cot_finetuning.sh

Streaming output truncated to the last 5000 lines.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Tra

# Merging Adapter Weights with Direct and COT fine-tuned model


Now we turn to the nextstep which is Merging the adapter weights to the top of fine tuned model for N 2-8.

# Direct + Adapter Merged Model


Preparing the script for Merging adapter weights with Direct fine-tuned models N2-8.

In [ ]:
import os
project_path = '/content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/'
os.chdir(project_path)
print(f"Current working directory: {os.getcwd()}")

Current working directory: /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs


In [ ]:
%%writefile scripts/ft/merge_adapter_direct.sh
#!/bin/bash

# Ensure the script stops if any command fails
set -e

# Define the base model (Phi-3)
BASE_MODEL_PATH="microsoft/Phi-3-mini-4k-instruct"

# Define the base directory where your fine-tuned DIRECT adapters are saved
ADAPTER_BASE_DIR="./Models/Intermd/Phi3/Direct"

# Define the parent directory where you want to save your merged DIRECT models
MERGED_MODELS_OUTPUT_DIR="./Models/Merged/Phi3/Direct"

# Create the output directory if it doesn't exist
mkdir -p "${MERGED_MODELS_OUTPUT_DIR}"

# Define the N values to iterate through (from 2 to 8 people)
N_VALUES=(2 3 4 5 6 7 8)

echo "Starting DIRECT model merging process..."

for N in "${N_VALUES[@]}"; do
    echo "--- Merging DIRECT model for ${N} people ---"

    # Construct the path to the adapter for the current N
    ADAPTER_PATH="${ADAPTER_BASE_DIR}/finetuned_people${N}/final_model"

    # Construct the target path for the merged model for the current N
    TARGET_MODEL_PATH="${MERGED_MODELS_OUTPUT_DIR}/merged_people${N}"

    # Check if the adapter path exists
    if [ -d "$ADAPTER_PATH" ]; then
        echo "Merging adapter: ${ADAPTER_PATH} into base model: ${BASE_MODEL_PATH}"
        echo "Saving merged model to: ${TARGET_MODEL_PATH}"

        python merge.py \
            --base_model_path "${BASE_MODEL_PATH}" \
            --adapter_path "${ADAPTER_PATH}" \
            --target_model_path "${TARGET_MODEL_PATH}"
        echo "--- Merging for ${N} people completed ---"
    else
        echo "Warning: Adapter path not found for ${N} people: ${ADAPTER_PATH}. Skipping merge for this N."
    fi
    echo "" # Add a blank line for readability
done

echo "All specified DIRECT model merges attempted."

Overwriting scripts/ft/merge_adapter_direct.sh


In [ ]:
# Make the merge script executable
!chmod +x scripts/ft/merge_adapter_direct.sh

# Run the merge script to combine adapters with the base model
!bash scripts/ft/merge_adapter_direct.sh

Starting DIRECT model merging process...
--- Merging DIRECT model for 2 people ---
Merging adapter: ./Models/Intermd/Phi3/Direct/finetuned_people2/final_model into base model: microsoft/Phi-3-mini-4k-instruct
Saving merged model to: ./Models/Merged/Phi3/Direct/merged_people2
2025-06-28 21:44:28.365335: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-28 21:44:28.382149: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751147068.403198   82359 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751147068.409683   82359 cuda_bl

# Chain of thought + Adapter Merged Model

In [ ]:
%%writefile scripts/ft/merge_adapter_cot.sh
#!/bin/bash

# Ensure the script stops if any command fails
set -e

# Define the base model (Phi-3)
BASE_MODEL_PATH="microsoft/Phi-3-mini-4k-instruct"

# Define the base directory where your fine-tuned CoT adapters are saved
ADAPTER_BASE_DIR="./Models/Intermd/Phi3/COT"

# Define the parent directory where you want to save your merged CoT models
MERGED_MODELS_OUTPUT_DIR="./Models/Merged/Phi3/COT"

# Create the output directory if it doesn't exist
mkdir -p "${MERGED_MODELS_OUTPUT_DIR}"

# Define the N values to iterate through (from 2 to 8 people)
N_VALUES=(2 3 4 5 6 7 8)

echo "Starting CoT model merging process..."

for N in "${N_VALUES[@]}"; do
    echo "--- Merging CoT model for ${N} people ---"

    # Construct the path to the adapter for the current N, assuming 'final_model' subdirectory
    ADAPTER_PATH="${ADAPTER_BASE_DIR}/finetuned_people${N}/final_model"

    # Construct the target path for the merged model for the current N
    TARGET_MODEL_PATH="${MERGED_MODELS_OUTPUT_DIR}/merged_people${N}"

    # Check if the adapter path exists
    if [ -d "$ADAPTER_PATH" ]; then
        echo "Merging adapter: ${ADAPTER_PATH} into base model: ${BASE_MODEL_PATH}"
        echo "Saving merged model to: ${TARGET_MODEL_PATH}"

        python merge.py \
            --base_model_path "${BASE_MODEL_PATH}" \
            --adapter_path "${ADAPTER_PATH}" \
            --target_model_path "${TARGET_MODEL_PATH}"
        echo "--- Merging for ${N} people completed ---"
    else
        echo "Warning: Adapter path not found for ${N} people: ${ADAPTER_PATH}. Skipping merge for this N."
    fi
    echo "" # Add a blank line for readability
done

echo "All specified CoT model merges attempted."

Writing scripts/ft/merge_adapter_cot.sh


In [ ]:

# Make the merge script executable
!chmod +x scripts/ft/merge_adapter_cot.sh

# Run the merge script to combine adapters with the base model
!bash scripts/ft/merge_adapter_cot.sh

Starting CoT model merging process...
--- Merging CoT model for 2 people ---
Merging adapter: ./Models/Intermd/Phi3/COT/finetuned_people2/final_model into base model: microsoft/Phi-3-mini-4k-instruct
Saving merged model to: ./Models/Merged/Phi3/COT/merged_people2
2025-06-29 12:21:39.944702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751199699.965642    4331 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751199699.972536    4331 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-29 12:21:39.995050: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critica

## Evaluations: First, we run the test on test dataset

In [ ]:
import os

# Set the environment variable for the Python kernel
os.environ["USE_VLLM"] = "1"

print(f"Python Kernel DEBUG: USE_VLLM is now set to '{os.environ.get('USE_VLLM')}'")

Python Kernel DEBUG: USE_VLLM is now set to '1'


In [ ]:
%cd /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/

/content


In [ ]:
import vllm
print(vllm.__version__)

In [ ]:
!pip show transformers

Name: transformers
Version: 4.53.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
%cd ~/.cache/huggingface/modules/transformers_modules/microsoft/Phi-3-mini-4k-instruct/


/root/.cache/huggingface/modules/transformers_modules/microsoft/Phi-3-mini-4k-instruct


In [ ]:
!grep -R "get_max_length" .


./0a67737cc96d2554230f90338b163bc6380a2a85/modeling_phi3.py:                max_cache_length = past_key_values.get_max_length()
grep: ./0a67737cc96d2554230f90338b163bc6380a2a85/__pycache__/modeling_phi3.cpython-311.pyc: binary file matches


In [ ]:
!nano /root/.cache/huggingface/modules/transformers_modules/microsoft/0a67737cc96d2554230f90338b163bc6380a2a85/modeling_phi3.py



/bin/bash: line 1: nano: command not found


In [ ]:
%%writefile scripts/eval/run_test.sh
export USE_VLLM=0
config="vllm"
max_token=2048
num_limit=100

models=(
    "microsoft/Phi-3-mini-4k-instruct" # Base model for comparison

    # Merged Direct Models
    "./Models/Merged/Phi3/Direct/merged_people2"
    "./Models/Merged/Phi3/Direct/merged_people3"
    "./Models/Merged/Phi3/Direct/merged_people4"
    "./Models/Merged/Phi3/Direct/merged_people5"
    "./Models/Merged/Phi3/Direct/merged_people6"
    "./Models/Merged/Phi3/Direct/merged_people7"
    "./Models/Merged/Phi3/Direct/merged_people8"

    # Merged CoT Models
    "./Models/Merged/Phi3/COT/merged_people2"
    "./Models/Merged/Phi3/COT/merged_people3"
    "./Models/Merged/Phi3/COT/merged_people4"
    "./Models/Merged/Phi3/COT/merged_people5"
    "./Models/Merged/Phi3/COT/merged_people6"
    "./Models/Merged/Phi3/COT/merged_people7"
    "./Models/Merged/Phi3/COT/merged_people8"
)

echo "Processing num_limit: $num_limit"
# Iterate over each model
for model in "${models[@]}";
do
    echo "Processing model: $model"

    # 0-Shot, With CoT
    python eval_kk.py --batch_size 2 --model ${model} --max_token ${max_token} --arch ${model} --ntrain 0 --config ${config} --use_vllm --cot --limit ${num_limit}  --split "test"
    # 0-Shot, Without CoT
    python eval_kk.py --batch_size 2 --model ${model} --max_token ${max_token} --arch ${model} --ntrain 0 --config ${config} --use_vllm  --limit ${num_limit}  --split "test"
    # 1-Shot, Without CoT
    python eval_kk.py --batch_size 2 --model ${model} --max_token ${max_token} --arch ${model} --ntrain 1 --config ${config} --use_vllm  --limit ${num_limit}  --split "test"
    # 1-Shot, With CoT
    python eval_kk.py --batch_size 2 --model ${model} --max_token ${max_token} --arch ${model} --ntrain 1 --config ${config} --use_vllm --cot --limit ${num_limit}  --split "test"

done

# The following section is for a specific Llama model and no-linebreak,
# and is commented out as it's likely not needed for your current task.
# model="meta-llama/Meta-Llama-3-8B"
# python eval_kk.py --no_linebreak --batch_size 8 --model ${model} --max_token ${max_token} --arch ${model} --ntrain 0 --config ${config} --use_vllm --cot --limit ${num_limit}  --split "test"
# python eval_kk.py --no_linebreak --batch_size 8 --model ${model} --max_token ${max_token} --arch ${model} --ntrain 0 --config ${config} --use_vllm  --limit ${num_limit}  --split "test"

Overwriting scripts/eval/run_test.sh


In [ ]:
import os
import re
import torch # Need torch for the padding logic

file_path = "/usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py"

if not os.path.exists(file_path):
    print(f"Error: modeling_attn_mask_utils.py not found at {file_path}.")
    print("Please ensure the transformers library is installed and this is the correct path.")
else:
    try:
        with open(file_path, 'r') as f:
            content = f.read()

        # Target line to insert the patch before.
        # This regex captures the leading whitespace (group 1) and the code itself (group 2).
        target_line_pattern = r"(\s*)(expanded_attn_mask = causal_4d_mask\.masked_fill\(expanded_attn_mask\.bool\(\), torch\.finfo\(dtype\)\.min\))"
        match = re.search(target_line_pattern, content)

        if match:
            # The indentation of the original line in the file (e.g., '        ')
            original_line_indentation = match.group(1)

            # The patch logic as a raw multi-line string.
            # Important: The internal indentation here is relative to the FIRST non-empty line of this string.
            patch_logic_raw = """
            # --- START PATCH FOR PHI-3 ATTENTION MASK BUG ---
            if expanded_attn_mask.shape[-1] != causal_4d_mask.shape[-1]:
                pad_len = causal_4d_mask.shape[-1] - expanded_attn_mask.shape[-1]
                if pad_len > 0:
                    # Pad expanded_attn_mask with True (1s) to match causal_4d_mask's length
                    # This ensures newly generated tokens are not masked out by padding.
                    padding_tensor = torch.ones(
                        expanded_attn_mask.shape[:-1] + (pad_len,),
                        dtype=expanded_attn_mask.dtype, # Keep original bool dtype
                        device=expanded_attn_mask.device
                    )
                    expanded_attn_mask = torch.cat((expanded_attn_mask, padding_tensor), dim=-1)
                # If pad_len is negative, it means expanded_attn_mask is larger,
                # which is not the issue we're addressing here based on the error.
                # No action needed for pad_len <= 0 in this specific patch.
            # --- END PATCH FOR PHI-3 ATTENTION MASK BUG ---
            """

            # Normalize the patch logic by removing its *own* minimum leading whitespace,
            # then prepend the `original_line_indentation` to each line.

            patch_lines = patch_logic_raw.strip().split('\n')

            # Determine the minimum indentation within the raw patch_logic_raw itself
            min_patch_indent = float('inf')
            for line in patch_lines:
                if line.strip(): # Only consider non-empty lines
                    min_patch_indent = min(min_patch_indent, len(line) - len(line.lstrip()))

            if min_patch_indent == float('inf'): # Case where patch_logic_raw was empty or all whitespace
                min_patch_indent = 0

            formatted_patch_lines = []
            for line in patch_lines:
                if line.strip():
                    # Remove the base indentation from the raw patch line, then add the target indentation
                    adjusted_line = line[min_patch_indent:] # Slice to remove the raw snippet's own base indent
                    formatted_patch_lines.append(original_line_indentation + adjusted_line)
                else:
                    # For empty lines, just add the base indentation of the original line
                    formatted_patch_lines.append(original_line_indentation)

            # Join the formatted lines to create the final patch string
            formatted_patch_content = "\n".join(formatted_patch_lines)

            # Check if the patch is already present to avoid duplication
            if "# --- START PATCH FOR PHI-3 ATTENTION MASK BUG ---" in content:
                print(f"Patch already appears to be in {file_path}. Skipping modification.")
            else:
                # Use re.sub to replace the matched line with the patch content followed by the original line.
                # r"\n\1\2" inserts a newline, then the captured indentation (\1), then the captured original code (\2).
                # This ensures the original line is preserved with its exact original indentation below the patch.
                new_content = re.sub(
                    target_line_pattern,
                    formatted_patch_content + r"\n\1\2", # \1 is indent, \2 is the original line of code
                    content,
                    count=1 # Only replace the first occurrence
                )

                with open(file_path, 'w') as f:
                    f.write(new_content)
                print(f"Successfully applied attention mask patch to {file_path}.")
                print("Please re-run your main script now.")
        else:
            print("Could not find the target line for patch insertion in modeling_attn_mask_utils.py.")
            print("Please manually verify the file content or try to locate the insertion point.")

    except Exception as e:
        print(f"An error occurred during file modification: {e}")
        print("Please consider manually checking the file or retrying.")

Patch already appears to be in /usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py. Skipping modification.


In [ ]:
# Make the evaluation script executable
!chmod +x scripts/eval/run_test.sh

# Run the test set evaluation
!bash scripts/eval/run_test.sh

# Step 4: Moving onto training samples for 0 shot and 1 shot, first we execute 0 shot evaluations


In [ ]:
%pwd

'/content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs'

In [ ]:
!pip install vllm==0.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.9/146.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.1/37.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121

In [ ]:
import os
project_path = '/content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/scripts/eval'
os.chdir(project_path)
print(f"Current working directory: {os.getcwd()}")

Current working directory: /content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/scripts/eval


In [ ]:
try:
    import vllm
    print("✅ vLLM was successfully imported!")
except ImportError as e:
    print(f"❌ Failed to import vLLM. Error: {e}")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

✅ vLLM was successfully imported!


In [ ]:
!pip install transformers==4.39.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 96.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.0
    Uninstalling transformers-4.55.0:
      Successfully uninstalled transformers-4.55.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.21.0 requires transformers>=4.55.0, but you have transformers 4.39.3 which is incompatible.
vllm 0.5.1 requires tokenizers>=0.19.1, but you have tokenizers 0.15.2 which is incompatible.
vllm 0.5.1 requires transformers>=4.42.0, but yo

In [ ]:
!bash run_train_0shot.sh

Processing eval_train: 2
Processing model: microsoft/Phi-3-mini-4k-instruct
DEBUG (utils.py): os.environ.get('USE_VLLM') = '1'
DEBUG (utils.py): os.environ.get('USE_VLLM', '0') == '1' is True
Traceback (most recent call last):
  File "/content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/scripts/eval/../../eval_kk.py", line 14, in <module>
    from utils import load_jsonl, write_jsonl, batch_decode_vllm, init_seed, load_llm, VLLM_AVAILABLE
  File "/content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/utils.py", line 22, in <module>
    from hf import CasualLM
  File "/content/drive/MyDrive/RA_Work/Mem_Rsn_LLMs/hf.py", line 4, in <module>
    import flash_attn
  File "/usr/local/lib/python3.11/dist-packages/flash_attn/__init__.py", line 3, in <module>
    from flash_attn.flash_attn_interface import (
  File "/usr/local/lib/python3.11/dist-packages/flash_attn/flash_attn_interface.py", line 15, in <module>
    import flash_attn_2_cuda as flash_attn_gpu
ImportError: /usr/local/lib/python3.11/dist-packages/fla

i also triee the batch size reduction but cuda memory issue is happening